### Cross Validation

In [ ]:
import pandas as pd
import numpy as np

# ESG standards dictionary (unchanged)
ESG_STANDARDS = {
    'Absolute emissions scope（total）(mtCO2e)': {'unit': 'mtCO2e', 'range': (100, 1_000_000), 'type': float},
    'Anti-corruption training for employees': {'range': (0, 5), 'type': int},
    'Assurance of sustainability report': {'range': (0, 1), 'type': int},
    'Average training hours per employee (hours)': {'range': (0, 200), 'type': float},
    'Current employees by age groups (30 -50) (%)': {'unit': '%', 'range': (0, 100), 'type': float},
    'Current employees by gender (%)': {'unit': '%', 'range': (0, 100), 'type': float},
    'Fatalities': {'range': (0, 100), 'type': int},
    'Total energy consumptionn (MWh)': {'unit': 'MWh', 'range': (0, 1_000_000), 'type': float},
    'Total number of employees': {'range': (0, 1_000_000), 'type': int},
    'Total waste generated(mts)': {'unit': 'mts', 'range': (0, 1_000_000), 'type': float},
    'Total water consumption(m3)': {'unit': 'm3', 'range': (0, 1_000_000), 'type': float},
    'Women on the board (%)': {'unit': '%', 'range': (0, 100), 'type': float},
    'turnover by gender': {'range': (0, 50), 'type': float}
}

# Load labeled dataset (ground truth) and extracted data
labeled_data = pd.read_excel("../data/extract.xlsx")
extracted_data = pd.read_excel("../data/Cross.xlsx")

# Define unit conversion functions
def convert_units(value, current_unit, target_unit):
    if current_unit == target_unit:
        return value
    elif current_unit == 'GWh' and target_unit == 'kWh':
        return value * 1000
    # Add other conversions as needed
    else:
        raise ValueError(f"Unsupported unit conversion from {current_unit} to {target_unit}")

# Validation function with missing value handling and counts for valid/invalid metrics
def validate_extracted_data(extracted_data, labeled_data):
    validation_results = []
    metric_valid_counts = {metric: {'valid': 0, 'invalid': 0} for metric in ESG_STANDARDS}  # Initialize counts for each metric
    total_valid_cells = 0  # Counter for overall valid cells
    total_evaluated_cells = 0  # Counter for overall evaluated cells

    for idx, row in extracted_data.iterrows():
        company, year = row['Company Name'], row['Year']
        gt_row = labeled_data[(labeled_data['Company Name'] == company) & (labeled_data['Year'] == year)]

        row_result = {'Company Name': company, 'Year': year}

        # Validate each ESG metric
        for metric, standard in ESG_STANDARDS.items():
            extracted_value = row[metric]
            target_unit = standard.get('unit', None)
            extracted_unit = row.get(f"{metric} Unit", target_unit)
            gt_value = gt_row[metric].values[0] if not gt_row.empty else np.nan
            
            # Skip validation if ground truth value is missing
            if pd.isna(gt_value):
                row_result[f'{metric}_Validation'] = 'Not Applicable'
                continue

            # Increment total evaluated cells
            total_evaluated_cells += 1

            # Convert extracted value to the correct type, if possible
            try:
                extracted_value = standard['type'](extracted_value)  # Convert to the defined type (e.g., float, int)
            except (ValueError, TypeError):
                row_result[f'{metric}_Format_Valid'] = False
                metric_valid_counts[metric]['invalid'] += 1  # Increment invalid count
                continue  # Skip further checks if conversion fails

            # Convert ground truth value if needed
            try:
                gt_value = standard['type'](gt_value)  # Ensure ground truth is also in the correct type
            except (ValueError, TypeError):
                row_result[f'{metric}_Format_Valid'] = False
                metric_valid_counts[metric]['invalid'] += 1  # Increment invalid count
                continue  # Skip further checks if conversion fails

            # Convert extracted value to the standard unit if both units are specified
            if target_unit and extracted_unit:
                extracted_value = convert_units(extracted_value, extracted_unit, target_unit)
            
            # Check if value is within the valid range and exact match
            is_within_range = standard['range'][0] <= extracted_value <= standard['range'][1]
            is_exact_match = extracted_value == gt_value

            # Record validation results
            row_result[f'{metric}_Range_Valid'] = is_within_range
            row_result[f'{metric}_Format_Valid'] = isinstance(extracted_value, standard['type'])
            row_result[f'{metric}_Exact_Match'] = is_exact_match

            # Update counts based on validation results
            if is_within_range and isinstance(extracted_value, standard['type']) and is_exact_match:
                metric_valid_counts[metric]['valid'] += 1
                total_valid_cells += 1  # Increment valid cell count
            else:
                metric_valid_counts[metric]['invalid'] += 1

        validation_results.append(row_result)

    # Compile results into a DataFrame
    validation_df = pd.DataFrame(validation_results)

    # Create a summary of valid/invalid counts per metric
    validation_summary = pd.DataFrame(metric_valid_counts).T
    validation_summary.columns = ['Valid Count', 'Invalid Count']

    # Calculate proportion of valid outputs per metric
    validation_summary['Proportion Valid'] = validation_summary['Valid Count'] / (validation_summary['Valid Count'] + validation_summary['Invalid Count'])

    # Calculate overall percentage of valid cells
    overall_valid_percentage = (total_valid_cells / total_evaluated_cells * 100) if total_evaluated_cells > 0 else 0

    return validation_df, validation_summary, overall_valid_percentage

# Run the validation
validation_df, validation_summary, overall_valid_percentage = validate_extracted_data(extracted_data, labeled_data)

# Print validation results and summary
print("Validation Results:\n", validation_df)
print("\nValidation Summary (Valid/Invalid Counts and Proportion of Valid Outputs per Metric):\n", validation_summary)
print(f"\nOverall Percentage of Valid Data: {overall_valid_percentage:.2f}%")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2185584173.py, line 22)